## Intro to Gnip
Gnip is a social media data aggregator that was purchased by Twitter in May 2014. We are the data pipeline to customers in over 40 countries who serve over 95% of the companies in the Fortune 500.  

## Data Science At Gnip
Gnip's data science team consists of five people with physcis and math backgrounds. We often get to work with enterprise level clients to find insights and develop solutions. Our team is focused on sharing our learning and we try to contribute to the open source community. 

With it's many libraries and continual development, Python continues to be our data science team's main language of choice. However, our team writes in serval languages. A typical workflow might start with simply using curl to pull in new data from Gnip's pipeline or else we might launch MapReduce jobs using Pig or Scala. We then massage the data using Python into a format that allows us to explore ideas and sometimes apply machine learning techniques. Depending on the project goals and size of data, we might make use SQL, R, or several Python libraries to find insights. 

We often build quick exploratory visualizations and share them inside IPython notebooks using Pandas and R. These notebooks allow us to easily create iterations, compare results, edit code on the fly, and even write explanations in markdown. We also regularly use R-studio, but Python based plotting tools (even those that allow us to directly write in R) are becomming more common. As IPython develops, I hope we'll see more opportunities to integrate with D3 - a favorite javascript library for building custom, interactive, web based visualizations.

Most of the data we explore is visualized using these graphs:
* Line graph (often used with time series)
* Bar graph (stacked, grouped, histogram)
* Scatter plot (often used to vary dimensions of color, label, and size)
* Cumulative Distribution Function plot 
* Probability Density Function plot 

Altering these visulaizations requires some know-how in these areas:
* Buckets (flexible aggregation techniques)
* Dates (slicing time series)
* Scales (representing regions of the data)

More advanced visualizations that we sometimes use:
* Choropleth (maps that tie geographic location to a scale)
* Networks (relationships among nodes)
* Hexbin (density plot)

## Team Culture & Visualization
Today we'll explore how team culture affects visualizations. As an example, we'll investigate Gnip's data science team.

This team holds two weekly events that promote learning: a quick standup and a weekly 45 mintue lecture series. The weekly tips & tricks standup generally showcases short snippets of code, command line utilities, or vim tricks that can advance the team's efficiency. The lecture series is a more indepth look at topic one member on the team is learning. The lecture is followed by 15 minutes of discussion. Both of these opportunities allow individuals on the team be  vulnerable with their expertise on a new topic. 

The weekly lectures and standups stabilize a share interest on our team: growth is necessary and desirable for all. The knowledge we build is something to which all parties contribute for the benefit of themselves and the team. 

These action dramatically impact our concept of an interative visualization process. 


In [ ]:
from IPython.display import Image
Image(filename='imgs/panda-sleep.jpg',width="80%") 

## Example 1: Pandas for Date Buckets
We find the pandas library useful and use it regularly. This specific example will focus on a common task for a data scientist: counting stuff. We will bucket the data and then count the number of events in each bucket.

In [ ]:
# get data
#with open("data/sample1000_withHeader_zerosEdited.csv") as f:
#   x = f.readlines()
#for line in x[0:5]:
#    print line.strip()

In [ ]:
# Load requirements 
import pandas as pd

# Make place inline as opposed to a separate window.
%matplotlib inline

#### Reading data
Let's start by opening a sample data file using Pandas. Typically, we would be gathering raw JSON, but here we have some previously parsed data to simplify the process. To understand the code below please see the [pd.read_csv() documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html). 

In [ ]:
# get data
df1 = pd.read_csv("data/sample1000_withHeader_zerosEdited.csv"
                      , sep = ","
                      , header = 0
                      , parse_dates = ["timeStamp"]
                      , encoding =  'utf-8'
                )
print type(df1)
print "df:\n{}".format(df1.tail())
# get data



#### Buckets
First, we have to decide on the size of our bucket. Since we're working with time series, we can focus on slices like day & hour. Below we have two methods for slicing our data. Each method produces different output, which will affect decisions that we'll make downstream. We can review a few conversations about [method 1](http://stackoverflow.com/questions/16266019/python-pandas-group-datetime-column-into-hour-and-minute-aggregations) and [method 2](https://github.com/pydata/pandas/issues/3791). Notice that the second method provides use with added information while preserving the date objects. 

In [ ]:
# order dates
df1['timeStamp'] = df1.timeStamp.order()

# bucket data
## method 1
g1 = df1.groupby([df1.timeStamp.map(lambda t: t.day), df1.timeStamp.map(lambda t: t.hour)]).size()

## method 2
df2 = df1
g2 = df2.set_index("timeStamp").groupby([pd.TimeGrouper('1D'),pd.TimeGrouper('1H')]).size()

print "----------\nMethod 1\n----------\n{}\n\n\n\n----------\nMethod 2\n----------\n{}".format(g1.head(),g2.head())

#### Iteration 1: the ol' .plot() method
Pandas methods sometimes seems like magic, which can be nice for a quick glance at the data. We can review documentation for [pandas.series.plot](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.plot.html). 

In [ ]:
# glance at g2
print g2.head()

# dirty g2 graph
g2.plot(rot=90, figsize=(10,5))

## Example 2: R for Date Buckets
Let's clean this graph up a bit using ggplot in R. First, we'll load the data into python lists, build a pandas dataframe, and then pass this information to R whence we shall use the ggplot library to make a pretty graph. See the [yhat's explanation](http://ggplot.yhathq.com/how-it-works.html). 

In [ ]:
# import requirments 
import rpy2
%load_ext rpy2.ipython
%R require("ggplot2")

# gather data for axes
dates = []
counts = g2.values
for tup in g2.keys().values:
    dates.append(tup[1])
#dir(g2)
#dir(g2.index)

# quick data check
print len(dates) == len(counts)
print "\ndates:\n{}\n\ncounts:\n{}".format(dates[:5],counts[:5])


In [ ]:
# create dataframe with dates and counts 
dfR = pd.DataFrame({
        "xValues":dates,
        "yValues":counts
    })

# pass dfR to R
%Rpush dfR

#### Iteration 2: ggplot 
Changing the basic aesthetics and adding additional layers for template based graphs, ggplot is an R library packs some real explanatory power. See how it works from [yhat's perspecetive](http://ggplot.yhathq.com/how-it-works.html). Below you'll see the `%R` with a few paramaters. This code is explicitly R, not python; so when the cell executes, we see the results similar to a plot in R-studio.

In [ ]:
%%R -w 900 -h 480 -u px
#line graph in R
p1 <- ggplot(data=dfR) + geom_line(aes(x=xValues, y=yValues)) + ggtitle("Filtered Tweets") + xlab("") + ylab("Hourly Counts") + theme (legend.position="none", text = element_text(size=20))
print(p1)

##Example 3: D3 for Date Buckets
This example allows us to transform matplotlib plots to HTML. The advantage here is interactivity. Notice that we use the same code from example 1, but we gain the ability to zoom.

In [ ]:
###
### FIX X AXIS DATES
###
import mpld3
import matplotlib.pyplot as plt
mpld3.enable_notebook()


#### Iteration 3: D3
A simple matplotlib graph with some added functionality. 

In [ ]:
g2.plot(rot=90, figsize=(10,5))

## Example 4: D3 extended

In [12]:
#...comming soon?